In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import time

### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
          .load("abfss://bronze@datalakete.dfs.core.windows.net/customers")
df.display()

In [0]:
df = df.withColumn("domains", split(col("email"), "@")[1])\
    .drop("_rescued_data")
df.display()

In [0]:
df.groupBy("domains").agg(count("customer_id").alias("total_customers")).sort("total_customers",ascending = False).display()

In [0]:
df_gmail = df.filter(col("domains") == "gmail.com")
df_gmail.display()
time.sleep(5)

df_gmail = df.filter(col("domains") == "yahoo.com")
df_gmail.display()
time.sleep(5)

df_gmail = df.filter(col("domains") == "hotmail.com")
df_gmail.display()
time.sleep(5)

In [0]:
df = df.withColumn("full_name", concat(col("first_name"),lit(" "),col("last_name")))\
       .drop("first_name","last_name")

df.display()

### **Silver : Data writing in delta**

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@datalakete.dfs.core.windows.net/customers")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_av_cata.silver.customers
USING DELTA
LOCATION "abfss://silver@datalakete.dfs.core.windows.net/customers"

In [0]:
%sql
select * from databricks_av_cata.silver.customers